In [53]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import asyncio
import joblib

In [54]:
!pwd

/Users/yoshihiroashihara/VFMcalcProj


In [55]:
!ls -la

total 2504
drwxr-xr-x@  27 yoshihiroashihara  staff      864 Aug  1 16:13 .
drwxr-x---+ 145 yoshihiroashihara  staff     4640 Aug  1 15:05 ..
-rw-r--r--@   1 yoshihiroashihara  staff     6148 Aug  2 16:57 .DS_Store
drwxr-xr-x   15 yoshihiroashihara  staff      480 Aug  2 12:38 .git
-rw-r--r--    1 yoshihiroashihara  staff     3078 Jul 30 08:56 .gitignore
-rw-r--r--    1 yoshihiroashihara  staff        7 Aug  1 01:13 .python-version
-rw-r--r--@   1 yoshihiroashihara  staff     1486 Jul 27 17:37 BOJ_ExpInflRate_down.py
-rw-r--r--    1 yoshihiroashihara  staff      506 Aug  2 17:09 Final_Inputs.pkl
-rw-r--r--    1 yoshihiroashihara  staff     4727 Aug  2 17:08 Final_Inputs.py
-rw-r--r--@   1 yoshihiroashihara  staff     7021 Jul 27 17:45 Flet_inputDialogTest.py
-rw-r--r--@   1 yoshihiroashihara  staff     5444 Aug  2 16:10 Flet_inputDialogTest_class.py
-rw-r--r--@   1 yoshihiroashihara  staff     6923 Jul 27 14:21 Flet_kinriselect _test.py
-rw-r--r--@   1 yoshihiroashihara  staff      511

In [56]:
results =joblib.load('results.pkl')
res_PSC_LCC = joblib.load('res_PSC_LCC.pkl')
final_inputs = joblib.load('Final_Inputs.pkl')
initial_inputs = joblib.load('Initial_Inputs.pkl')

In [57]:
#final_inputs.keys()
initial_inputs.keys()
#proj_years = final_inputs['proj_years']

dict_keys(['mgmt_type', 'proj_ctgry', 'proj_type', 'proj_years', 'const_years', 'kijun_kinri', 'chisai_kinri', 'zei_modori', 'lg_spread', 'zei_total', 'growth', 'kitai_bukka', 'shisetsu_seibi', 'ijikanri_unnei', 'reduc_shisetsu', 'reduc_ijikanri', 'pre_kyoukouka', 'kisai_jutou', 'kisai_koufu', 'zeimae_rieki', 'SPC_keihi', 'hojo'])

In [58]:
print(initial_inputs['kisai_jutou'])
print(initial_inputs['kisai_koufu'])
print(initial_inputs['hojo'])
print(initial_inputs['zei_modori'])
print(initial_inputs['mgmt_type'])
print(initial_inputs['proj_years'])
print(initial_inputs['const_years'])
print(final_inputs['kisai_jutou'])
print(final_inputs['kisai_koufu'])
print(final_inputs['proj_years'])
print(final_inputs['const_years'])
print(res_PSC_LCC['proj_years'])
print(res_PSC_LCC['const_years'])

75.0
30.0
30.0
8.4
市町村
30
3
75.0
30.0
24
3
20
3


In [59]:
const_years = res_PSC_LCC['const_years']
ijikanri_years = res_PSC_LCC['ijikanri_years']
proj_years = res_PSC_LCC['proj_years']
discount_rate = res_PSC_LCC['discount_rate']
print(const_years)
print(ijikanri_years)
print(proj_years)

3
17
20


In [60]:
print(final_inputs['proj_years'])
print(final_inputs['const_years'])

24
3


In [61]:
PSC_const_discount_factor = [(1/(1-float(discount_rate)/100))** i for i in reversed(range(int(const_years)))]
LCC_discount_factor = [(1/(1+float(discount_rate)/100))** i for i in range(1, int(proj_years)+1)]
print(const_years)
print(discount_rate)
print(PSC_const_discount_factor)
print(proj_years)
print(LCC_discount_factor)

3
3.249724880382775
[1.0683057912353333, 1.0335887921389886, 1.0]
20
[0.9685255831514548, 0.9380418052188655, 0.908517486420045, 0.8799224283382681, 0.8522273830343652, 0.8254040231309968, 0.7994249128385055, 0.7742634798927144, 0.7498939883759659, 0.7262915123936026, 0.703431910578966, 0.681291801400835, 0.6598485392480489, 0.6390801912668521, 0.6189655149272711, 0.5994839362955758, 0.5806155289906021, 0.5623409938024133, 0.5446416389524509, 0.5274993609749866]


In [49]:
results['LCC_discount_factor']

[0.9685255831514548,
 0.9380418052188655,
 0.908517486420045,
 0.8799224283382681,
 0.8522273830343652,
 0.8254040231309968,
 0.7994249128385055,
 0.7742634798927144,
 0.7498939883759659,
 0.7262915123936026,
 0.703431910578966,
 0.681291801400835,
 0.6598485392480489,
 0.6390801912668521,
 0.6189655149272711,
 0.5994839362955758,
 0.5806155289906021,
 0.5623409938024133,
 0.5446416389524509,
 0.5274993609749866]

In [50]:
results['PSC_const_discount_factor']

[1.0683057912353333, 1.0335887921389886, 1.0]

In [51]:
results['LCC_discount_factor']

[0.9685255831514548,
 0.9380418052188655,
 0.908517486420045,
 0.8799224283382681,
 0.8522273830343652,
 0.8254040231309968,
 0.7994249128385055,
 0.7742634798927144,
 0.7498939883759659,
 0.7262915123936026,
 0.703431910578966,
 0.681291801400835,
 0.6598485392480489,
 0.6390801912668521,
 0.6189655149272711,
 0.5994839362955758,
 0.5806155289906021,
 0.5623409938024133,
 0.5446416389524509,
 0.5274993609749866]

In [32]:
JGB_rates = pd.read_csv('https://www.mof.go.jp/jgbs/reference/interest_rate/data/jgbcm_all.csv', encoding='Shift-JIS', header=1)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
JGB_rates.head()

In [ ]:
def dateConv(date):

    wareki = {
            "R":2018,
            "H":1988,
            "S":1925
            }
  
    date_els = date.split('.', 2)
    year_num = int(date_els[0][1:])
    era = date_els[0][0]
 
    year = year_num + wareki[era]

    month =int(date_els[1])
    day = int(date_els[2])

    dt = datetime(year, month, day)

    return dt



In [ ]:
JGB_rates["基準日"] = JGB_rates['基準日'].apply(dateConv)
JGB_rates = JGB_rates.set_index('基準日')
JGB_rates.replace('-', np.nan, inplace=True)

In [ ]:
JGB_rates.head()

In [ ]:
JGB_rates.info()

In [ ]:
JGB_rates['5年'].resample('Y').mean()

In [ ]:
JGB_rate_5_30 = JGB_rates[['5年', '10年', '15年', '20年', '25年', '30年']].dropna()
JGB_rate_5_30 = JGB_rate_5_30.astype(float, errors='raise')

In [ ]:
JGB_rate_5_30_12M = JGB_rate_5_30.resample('M').mean().tail(12)
JGB_rate_5_30_12M

In [ ]:
JGB_rate_5_30_12M.mean()